In [ ]:
import tensorflow as tf
from tensorflow.python.layers.core import Dense
import tensorflow.contrib.keras as keras
from keras.preprocessing.sequence import pad_sequences
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
import time
import random
from PIL import Image
from trec_eval import trec_eval
import nltk
import copy

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
config = tf.ConfigProto()

In [ ]:
random.seed(1)
np.random.seed(1)

# Useful Functions

In [ ]:
def negative_samples(num_samples,toplist,downlist,combinationlist):
    sampledata = []
    num = 0
    while num < num_samples:
        top = random.sample(toplist,1)[0]
        down = random.sample(downlist,1)[0]
        if top+down not in combinationlist:
            sampledata.append((top,down,-1))
            num += 1
    return sampledata

In [ ]:
def batch_to_input(batch,imglist,topidlist,downidlist):
    img1 = []#for top
    img2 = []#for down
    img1id = []
    img2id = []
    label = []
    for instance in batch:
        img1.append(imglist[instance[0]])
        img2.append(imglist[instance[1]])
        img1id.append(topidlist[instance[0]])
        img2id.append(downidlist[instance[1]])
        commentid = instance[2]
        if commentid == -1:
            label.append([1,0])
        else:
            label.append([0,1])
    return np.array(img1),np.array(img2),np.array(img1id),np.array(img2id),np.array(label)

In [ ]:
def get_batches(data,batch_size,toplist,downlist,combinationlist,imglist,topidlist,downidlist):
    datacopy = copy.copy(data)
    datacopy.extend(negative_samples(len(datacopy),toplist,downlist,combinationlist))
    random.shuffle(datacopy)
    for batch_i in range(0,len(datacopy)//batch_size+1):
        start_i = batch_i*batch_size
        batch = datacopy[start_i:start_i+batch_size]         
        yield batch_to_input(batch,imglist,topidlist,downidlist)

In [ ]:
def build_evaluation_batch(fixitem,itemlist,state,imglist,topidlist,downidlist):
    img1 = []
    img2 = []
    img1id = []
    img2id = []
    if state == 0:#top,downs
        for item in itemlist:
            img1.append(imglist[fixitem])
            img2.append(imglist[item])
            img1id.append(topidlist[fixitem])
            img2id.append(downidlist[item])
    if state == 1:#down,tops
        for item in itemlist:
            img1.append(imglist[item])
            img2.append(imglist[fixitem])
            img1id.append(topidlist[item])
            img2id.append(downidlist[fixitem])
    return np.array(img1),np.array(img2),np.array(img1id),np.array(img2id)

In [ ]:
def accuracy(label,prediction):
    return (label.argmax(axis=1) == prediction.argmax(axis=1)).sum()/len(label)

In [ ]:
def prepare_evaluation(data_path):  
    with open(data_path,'r') as f:
        content = f.readlines()
    data = {}
    orderlist = []
    labellist = {}
    query_number = 0
    for line in content:
        line = line[:-2].split('\t')
        if data.get(line[0]) != None:
            data[line[0]].append(line[1])
        else:
            data[line[0]] = [line[1]] 
            labellist[query_number] = {}
            query_number += 1
            orderlist.append(line[0])
        if int(line[2]) == 1:
            labellist[query_number-1][line[1]] = 1
        else:
            labellist[query_number-1][line[1]] = 0
    return data,orderlist,labellist

In [ ]:
def trec_evaluation(qrel_file_path,trec_file_path,trec):
    with open(trec_file_path,'w') as f:
        i = 0
        while i < len(trec):
            j = 0 
            while j < len(trec[i]):
                f.write(str(i)+' '+'Q0 '+trec[i][j][0]+' '+str(j+1)+' '+str(trec[i][j][1])+' '+'Exp'+'\n')#注意排序从1开始
                j += 1
            i += 1   
    result = trec_eval(qrel_file_path,trec_file_path)
    print(result)
    return result

In [ ]:
def auc_evaluation(labellist,trec):
    query_number = 0
    record = []
    while query_number < len(trec):
        negative = 0
        temp = []
        for combination in trec[query_number]:
            if labellist[query_number][combination[0]] == 1:
                temp.append(negative)
            else:
                negative += 1
        record.extend([(negative-val)/float(negative) for val in temp])
        query_number += 1
    auc = np.array(record).mean()
    print(auc)
    return auc

# Prepare Datasets

In [ ]:
toplist = []
topidlist = {}
with open('dataset/toplist.dat','r') as f:#in toplist, the first col is img_name of top, the second col is comments_index
    content = f.readlines()
for line in content:
    line = line[:-2]
    toplist.append(line)
    topidlist[line] = len(topidlist)
toplist

In [ ]:
topidlist

In [ ]:
downlist = []
downidlist = {}
with open('dataset/downlist.dat','r') as f:#in downlist, the first col is img_name of down(i.e. bottom), the second col is comments_index
    content = f.readlines()
for line in content:
    line = line[:-2]
    downlist.append(line)
    downidlist[line] = len(downidlist)
downlist

In [ ]:
downidlist

In [ ]:
combinationlist = set()
with open('dataset/combinationlist.dat','r') as f:#in combinationlist, the first col is img_name of top, the second col is img_name of down(i.e. bottom), the third col is comments_index    
    content = f.readlines()
for line in content:
    line = line[:-2].split('\t')
    combinationlist.add(line[0]+line[1])
combinationlist

In [ ]:
imglist = {}
for img_idx in toplist:
    img = Image.open('img/'+img_idx+'.jpg')
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = np.array(img)
    img = img/255.0
    imglist[img_idx] = img
for img_idx in downlist:
    img = Image.open('img/'+img_idx+'.jpg')
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = np.array(img)
    img = img/255.0
    imglist[img_idx] = img
imglist

# Build Model

In [ ]:
def get_input():
    img1 = tf.placeholder(tf.float32,[None,150,150,3],'img1')
    img2 = tf.placeholder(tf.float32,[None,150,150,3],'img2')
    img1id = tf.placeholder(tf.int32,[None,],'img1id')
    img2id = tf.placeholder(tf.int32,[None,],'img2id')
    label = tf.placeholder(tf.float32,[None,2],'label')
    learning_rate = tf.placeholder(tf.float32,[],name='learning_rate')
    keep_prob = tf.placeholder(tf.float32,[],name='keep_prob')
    return img1,img2,img1id,img2id,label,learning_rate,keep_prob

In [ ]:
def extractor(img):
    conv1 = keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu',data_format='channels_last',kernel_initializer='glorot_normal')(img)
    conv2 = keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu',data_format='channels_last',kernel_initializer='glorot_normal')(conv1)
    pool1 = keras.layers.MaxPool2D(pool_size=(16,16),padding='same')(conv1)
    pool2 = keras.layers.MaxPool2D(pool_size=(16,16),padding='same')(conv2)
    concat = keras.layers.Concatenate(axis=-1)([pool1,pool2])
    globalpool = keras.layers.GlobalAveragePooling2D()(concat)
    return concat,globalpool

In [ ]:
def image_to_image_attention(conv,globalpool):#conv=[batch_size,14,14,64]，globalpool=[batch_size,64] 
    weights1 = tf.get_variable('weights1',shape=[64,64],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    weights2 = tf.get_variable('weights2',shape=[64,64],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    weights3 = tf.get_variable('weights3',shape=[64,1],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    attn_from = tf.matmul(globalpool,weights1)#attn_form=[batch_size,64]
    features = keras.layers.Reshape([-1,64])(conv)#features=[batch_size,196,64] 
    attn_to = tf.matmul(tf.reshape(features,[-1,64]),weights2)#tf.reshape(features,[-1,64])=[batch_size*196,64]，attn_to=[batch_size*196,64]
    attn_from  = tf.expand_dims(attn_from,1)#attn_from=[batch_size,1,64]
    attn_to = tf.reshape(attn_to,tf.shape(features))#attn_to=[batch_size,196,64] 
    attn_logit = tf.add(attn_from,attn_to)#attn_logit=[batch_size,196,64]
    attn_logit = tf.reshape(attn_logit,[-1,64])#attn_logit=[batch_size*196,64]
    attn_logit = tf.tanh(attn_logit)
    attn_weight = tf.matmul(attn_logit,weights3)#attn_weight=[batch_size*196,1]
    attn_weight = tf.reshape(attn_weight,shape=[tf.shape(conv)[0],tf.shape(conv)[1]*tf.shape(conv)[2]])#attn_weight=[batch_size,196]
    attn_weight = tf.nn.softmax(attn_weight,name='attention_img2img')
    attn_weight = tf.expand_dims(attn_weight,-1)#attn_weight=[batch_size,196,1]
    attn_conv = tf.multiply(features,attn_weight)#attn_conv=[batch_size,196,64]
    attn_conv = tf.reduce_sum(attn_conv,axis=1)#attn_conv=[batch_size,64]
    return features,attn_conv#e=v^Ttanh(W1s+W2h)，a=softmax(e)

In [ ]:
def img2vec(conv):
    extractor_output = keras.layers.Dense(300,activation='relu',kernel_initializer='glorot_normal')(conv)
    return extractor_output

In [ ]:
def img_embedding(img1id,img2id):
    top_embedding_matrix = tf.get_variable('top_embedding_matrix',shape=[len(toplist),embedding_size],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    down_embedding_matrix = tf.get_variable('down_embedding_matrix',shape=[len(downlist),embedding_size],initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    img1_embedding = tf.nn.embedding_lookup(top_embedding_matrix,img1id)
    img2_embedding = tf.nn.embedding_lookup(down_embedding_matrix,img2id)
    return img1_embedding,img2_embedding

In [ ]:
def classifier(extractor_output,keep_prob):
    dense = keras.layers.Dense(256,activation='relu',kernel_initializer='glorot_normal')(extractor_output)
    dropout = tf.nn.dropout(dense,keep_prob)
    classifier_output = keras.layers.Dense(2,activation='softmax',kernel_initializer='glorot_normal')(dropout)
    return classifier_output

In [ ]:
def loss(classifier_output,label):
    classifier_loss = tf.reduce_mean(tf.contrib.keras.losses.categorical_crossentropy(label,classifier_output),name='classifier_loss')
    tv = tf.trainable_variables()
    reg_loss = tf.reduce_sum([tf.nn.l2_loss(v) for v in tv])
    loss = tf.add(classifier_loss,0.00001*reg_loss,name='loss')
    return loss

In [ ]:
def optimizer(loss,learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss)
    capped_gradients = [(tf.clip_by_value(grad,-5.,5.),var) for grad,var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    return train_op

In [ ]:
def prediction(classifier_output):
    prediction = tf.identity(classifier_output,name='prediction')
    return prediction

In [ ]:
embedding_size = 300
train_graph = tf.Graph()
with train_graph.as_default():
    tf.set_random_seed(1)
    with tf.name_scope('inputs'):
        img1,img2,img1id,img2id,label,learning_rate,keep_prob = get_input()
    with tf.name_scope('extractor'):
        with tf.variable_scope('extractor'):
            conv_img1,globalpool_img1 = extractor(img1)
        with tf.variable_scope('extractor',reuse=True):
            conv_img2,globalpool_img2 = extractor(img2)
        with tf.variable_scope('image_to_image_attention'):
            features_img1,attn_conv_img1 = image_to_image_attention(conv_img1,globalpool_img2)
        with tf.variable_scope('image_to_image_attention',reuse=True):
            features_img2,attn_conv_img2 = image_to_image_attention(conv_img2,globalpool_img1)
        with tf.variable_scope('img2vec'):
            extractor_output_img1 = img2vec(attn_conv_img1)
        with tf.variable_scope('img2vec',reuse=True):
            extractor_output_img2 = img2vec(attn_conv_img2)
        with tf.variable_scope('img_embedding'):
            img1_embedding,img2_embedding = img_embedding(img1id,img2id)
        encoder_output = tf.concat([features_img1,features_img2],axis=1,name='encoder_output')
        extractor_output = tf.concat([extractor_output_img1,extractor_output_img2,img1_embedding,img2_embedding],axis=1,name='extractor_output')
    with tf.name_scope('classifier'):
        classifier_output = classifier(extractor_output,keep_prob)
    with tf.name_scope('prediction'):
        prediction = prediction(classifier_output)
    with tf.name_scope('loss'):
        loss = loss(classifier_output,label)
    with tf.name_scope('optimizer'): 
        train_op = optimizer(loss,learning_rate)   

# Train Model

In [ ]:
with open('dataset/traindata.dat','r') as f:#in traindata, the first col is img_name of top, the second col is img_name of down(i.e. bottom)  
    content = f.readlines()
traindata = []
for line in content:
    line = line[:-2].split('\t')
    traindata.append((line[0],line[1],1))
traindata

In [ ]:
tops_qrel_file_path = 'evaluation/devdata_tops_qrel.dat'
tops_trec_file_path = 'evaluation/devdata_tops_trec.dat'
#downs_qrel_file_path = 'evaluation/devdata_downs_qrel.dat'
#downs_trec_file_path = 'evaluation/devdata_downs_trec.dat'

In [ ]:
data_path = 'dataset/devdata_tops.dat'
dev_tops_data,tops_orderlist,tops_labellist = prepare_evaluation(data_path)

In [ ]:
#data_path = 'dataset/devdata_downs.dat'
#dev_downs_data,downs_orderlist,downs_labellist = prepare_evaluation(data_path)

In [ ]:
lr = 0.001
epochs = 30
rate = 1.0
batch_size = 64

In [ ]:
cost_list = []
auc_tops = []
trec_evals_tops = []
#trec_evals_downs = []
#auc_downs = []

In [ ]:
checkpoint = 'checkpoint/trained_model.ckpt'
with tf.Session(graph=train_graph,config=config) as sess:
    writer = tf.summary.FileWriter('checkpoint/',sess.graph)
    saver = tf.train.Saver(max_to_keep=30)
    sess.run(tf.global_variables_initializer())
    print(time.localtime())
    classifier_loss = train_graph.get_tensor_by_name('loss/classifier_loss:0')
    for epoch in range(epochs):
        train_cost = 0
        temp_cost_list = []
        step = 0
        for _,(x_i1,x_i2,x_id1,x_id2,y_l) in enumerate(get_batches(traindata,batch_size,toplist,downlist,combinationlist,imglist,topidlist,downidlist)):
            _,cost = sess.run([train_op,classifier_loss],{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,label:y_l,learning_rate:lr,keep_prob:rate})
            train_cost += cost
            step += 1
            if step%1000 == 0:
                temp_cost_list.append(train_cost/step)
                print(str(train_cost/step)+' '+'pass!')
        temp_cost_list.append(train_cost/step)
        cost_list.append(temp_cost_list)
        print('Epoch {}/{} - Training Loss: {:.3f}'.format(epoch+1,epochs,train_cost/step))
        saver.save(sess,checkpoint,global_step=epoch+1)
        print('Model Trained and Saved')
        print(time.localtime())
        #validation      
        tops_trec = {}
        query_number = 0
        step = 0
        for top in tops_orderlist:
            downsoftop = dev_tops_data[top]
            probabilitylist = {}
            for batch_i in range(len(downsoftop)//batch_size+1):
                start_i = batch_i*batch_size
                downs = downsoftop[start_i:start_i+batch_size]
                x_i1,x_i2,x_id1,x_id2 = build_evaluation_batch(top,downs,0,imglist,topidlist,downidlist)
                seq_len = [30]*len(x_i1)
                prob = sess.run(prediction,{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,keep_prob:1.0})
                j = 0
                for down in downs:
                    probabilitylist[down] = prob[j][1]
                    j += 1 
                step += 1
                if step%1000 == 0:
                    print('pass!')
            tops_trec[query_number] = sorted(probabilitylist.items(),key=lambda item:item[1],reverse=True)
            del probabilitylist,downsoftop
            query_number += 1
        auc_tops.append(auc_evaluation(tops_labellist,tops_trec))
        trec_evals_tops.append(trec_evaluation(tops_qrel_file_path,tops_trec_file_path,tops_trec))
        del tops_trec
        '''
        downs_trec = {}
        query_number = 0
        step = 0
        for down in downs_orderlist:
            topsofdown = dev_downs_data[down]
            probabilitylist = {}
            for batch_i in range(len(topsofdown)//batch_size+1):
                start_i = batch_i*batch_size
                tops = topsofdown[start_i:start_i+batch_size]
                x_i1,x_i2,x_id1,x_id2 = build_evaluation_batch(down,tops,1,imglist,topidlist,downidlist)
                prob = sess.run(prediction,{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,keep_prob:1.0})
                j = 0
                for top in tops:
                    probabilitylist[top] = prob[j][1]
                    j += 1
                step += 1
                if step%1000 == 0:
                    print('pass!')
            downs_trec[query_number] = sorted(probabilitylist.items(),key=lambda item:item[1],reverse=True)
            del probabilitylist,topsofdown
            query_number += 1
        auc_downs.append(auc_evaluation(downs_labellist,downs_trec))
        trec_evals_downs.append(trec_evaluation(downs_qrel_file_path,downs_trec_file_path,downs_trec))
        del downs_trec
        '''
        #validation      
        print(time.localtime())

# Evaluate Model

In [ ]:
tops_qrel_file_path = 'evaluation/testdata_tops_qrel.dat'
tops_trec_file_path = 'evaluation/testdata_tops_trec.dat'
downs_qrel_file_path = 'evaluation/testdata_downs_qrel.dat'
downs_trec_file_path = 'evaluation/testdata_downs_trec.dat'

In [ ]:
data_path = 'dataset/testdata_tops.dat'#in testdata_tops, the first col is img_name of top, the second col is img_name of down(i.e. bottom), the third col is rel(1 relevant, 0 irrelevant)  
test_tops_data,tops_orderlist,tops_labellist = prepare_evaluation(data_path)

In [ ]:
data_path = 'dataset/testdata_downs.dat'#in testdata_downs, the first col is img_name of down(i.e. bottom), the second col is img_name of top, the third col is rel(1 relevant, 0 irrelevant) 
test_downs_data,downs_orderlist,downs_labellist = prepare_evaluation(data_path)

In [ ]:
print(time.localtime())
checkpoint = 'checkpoint/trained_model.ckpt-23'
test_graph = tf.Graph()
with tf.Session(graph=test_graph,config=config) as sess:
    loader = tf.train.import_meta_graph(checkpoint+'.meta')
    loader.restore(sess,checkpoint)
    img1 = test_graph.get_tensor_by_name('inputs/img1:0')
    img2 = test_graph.get_tensor_by_name('inputs/img2:0')
    img1id = test_graph.get_tensor_by_name('inputs/img1id:0')
    img2id = test_graph.get_tensor_by_name('inputs/img2id:0')
    keep_prob = test_graph.get_tensor_by_name('inputs/keep_prob:0')
    prediction = test_graph.get_tensor_by_name('prediction/prediction:0')
    batch_size = 64
    tops_trec = {}
    query_number = 0
    step = 0
    for top in tops_orderlist:
        downsoftop = test_tops_data[top]
        probabilitylist = {}
        for batch_i in range(len(downsoftop)//batch_size+1):
            start_i = batch_i*batch_size
            downs = downsoftop[start_i:start_i+batch_size]
            x_i1,x_i2,x_id1,x_id2 = build_evaluation_batch(top,downs,0,imglist,topidlist,downidlist)
            prob = sess.run(prediction,{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,keep_prob:1.0})
            j = 0
            for down in downs:
                probabilitylist[down] = prob[j][1]
                j += 1 
            step += 1
            if step%1000 == 0:
                print('pass!')
        tops_trec[query_number] = sorted(probabilitylist.items(),key=lambda item:item[1],reverse=True)
        del probabilitylist,downsoftop
        query_number += 1
    auc_evaluation(tops_labellist,tops_trec)
    trec_evaluation(tops_qrel_file_path,tops_trec_file_path,tops_trec)
    del tops_trec
    downs_trec = {}
    query_number = 0
    step = 0
    for down in downs_orderlist:
        topsofdown = test_downs_data[down]
        probabilitylist = {}
        for batch_i in range(len(topsofdown)//batch_size+1):
            start_i = batch_i*batch_size
            tops = topsofdown[start_i:start_i+batch_size]
            x_i1,x_i2,x_id1,x_id2 = build_evaluation_batch(down,tops,1,imglist,topidlist,downidlist)
            prob = sess.run(prediction,{img1:x_i1,img2:x_i2,img1id:x_id1,img2id:x_id2,keep_prob:1.0})
            j = 0
            for top in tops:
                probabilitylist[top] = prob[j][1]
                j += 1
            step += 1
            if step%1000 == 0:
                print('pass!')
        downs_trec[query_number] = sorted(probabilitylist.items(),key=lambda item:item[1],reverse=True)
        del probabilitylist,topsofdown
        query_number += 1
    auc_evaluation(downs_labellist,downs_trec)
    trec_evaluation(downs_qrel_file_path,downs_trec_file_path,downs_trec)
    del downs_trec
print(time.localtime())